In [1]:
'''
The content in notebook based on Ref but I try to convert them to Pyspark 
Ref: 
[1] Advanced Analytics with Spark, 2e
[2] https://github.com/novelari/advanced-analytics-spark
'''

'\nThe content in notebook based on Ref but I try to convert them to Pyspark \nRef: \n[1] Advanced Analytics with Spark, 2e\n[2] https://github.com/novelari/advanced-analytics-spark\n'

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as t
from pyspark.sql.types import StringType
from pyspark.sql.functions import split

In [3]:
config = SparkConf().setAll([('spark.driver.memory','10g'),('spark.executor.memory', '20g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3')])
# config.set("spark.driver.memory", "10g")
# config.set("spark.executer.memory", "20g")

In [4]:
sc = SparkContext(appName= "Recommendation_sys", conf = config)
# sc.stop()

In [5]:
spark = SparkSession.builder\
                    .master('local')\
                    .appName("SparkSession_appName")\
                    .getOrCreate()

In [6]:
main = "C:/Users/New/Downloads/Data science in Action/Data set for spark/advance analytic with spark/data/chapter 3/"
# Spark.SQL_DF
# rawUserArtistData_ss = spark.read.option("delimiter"," ")\
#                                  .option("inferSchema","true")\
#                                  .option("charset", "utf-8")\
#                                  .csv(main + "user_artist_data.txt").cache()

# rawArtistData_ss = spark.read.option("delimiter",'\t')\
#                           .option("inferSchema","true")\
#                           .option("charset", "utf-8")\
#                           .csv(main + "artist_data.txt").cache()

# rawArtistAlias_ss = spark.read.option("delimiter",'\t')\
#                           .option("inferSchema","true")\
#                           .option("charset", "utf-8")\
#                           .csv(main + "artist_alias.txt").cache()

# print(type(rawUserArtistData_ss)) # SPARK_SQL_DF

In [7]:
# call data with SparkContext
rawUserArtistData = sc.textFile(main + "user_artist_data.txt").cache()
rawArtistData = sc.textFile(main + "artist_data.txt").cache()
rawArtistAlias = sc.textFile(main + "artist_alias.txt").cache()

print(type(rawUserArtistData)) # RDD

<class 'pyspark.rdd.RDD'>


In [8]:
UAD = rawUserArtistData 
AD = rawArtistData
AA = rawArtistAlias

In [9]:
# prevent effect of Null value 
def Int_checker(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [10]:
Int_checker('a')
# => when we use with filter the non-INT data will be remove Automatically

False

In [12]:
print('AD')
AD.take(5)

AD


['1134999\t06Crazy Life',
 '6821360\tPang Nakarin',
 '10113088\tTerfel, Bartoli- Mozart: Don',
 '10151459\tThe Flaming Sidebur',
 '6826647\tBodenstandig 3000']

In [13]:
new_AD = AD.map(lambda each_row: each_row.split('\t'))\
            .filter(lambda x: x[0] and Int_checker(x[0]))\
            .map(lambda x: (int(x[0]), x[1].strip()))

# non-INT data will be remove Automatically
# remove all the leading and trailing spaces from a string
new_AD.take(5)

[(1134999, '06Crazy Life'),
 (6821360, 'Pang Nakarin'),
 (10113088, 'Terfel, Bartoli- Mozart: Don'),
 (10151459, 'The Flaming Sidebur'),
 (6826647, 'Bodenstandig 3000')]

In [14]:
print('AA')
AA.take(5)

AA


['1092764\t1000311',
 '1095122\t1000557',
 '6708070\t1007267',
 '10088054\t1042317',
 '1195917\t1042317']

In [15]:
'''Mapping badID to goodID'''
new_AA = AA.map(lambda a: a.split('\t'))\
           .filter(lambda a: a[0] and Int_checker(a[0]))\
           .filter(lambda a: a[1] and Int_checker(a[1]))\
           .map(lambda a: (int(a[0]), int(a[1])))\
           .collectAsMap()

# Dict for key = 1092764
sc.broadcast(new_AA).value[1092764]

1000311

In [16]:
print('UAD')
UAD.take(5)

UAD


['1000002 1 55',
 '1000002 1000006 33',
 '1000002 1000007 8',
 '1000002 1000009 144',
 '1000002 1000010 314']

In [17]:
'''--------------------------------------------------RDD-Based API----------------------------------------------------------'''
from pyspark.mllib.recommendation import ALS, Rating

def buildRatings(rawUserArtistData, bArtistAlias):
    def getArtistRating(line):
        (userID, artistID, count) = map(lambda x: int(x), line.split(' '))
        try:
            finalArtistID = bArtistAlias.value[artistID]
        except KeyError:
            finalArtistID = artistID
        return Rating(userID, finalArtistID, count)

    return rawUserArtistData.map(lambda line: getArtistRating(line))

In [18]:
artistByID = new_AD
artistAlias = new_AA

In [21]:
bArtistAlias = sc.broadcast(new_AA)
trainData = buildRatings(UAD, bArtistAlias)
trainData.take(5)

[Rating(user=1000002, product=1, rating=55.0),
 Rating(user=1000002, product=1000006, rating=33.0),
 Rating(user=1000002, product=1000007, rating=8.0),
 Rating(user=1000002, product=1000009, rating=144.0),
 Rating(user=1000002, product=1000010, rating=314.0)]

In [22]:
model = ALS.trainImplicit(ratings=trainData, 
                          rank=50, 
                          iterations=10, 
                          lambda_=0.01, 
                          alpha=40.0)

In [23]:
type(model)

pyspark.mllib.recommendation.MatrixFactorizationModel

In [24]:
# trainData.unpersist()
# 50 User Features for 5 user
model.userFeatures().mapValues(lambda v: ", ".join( map(lambda x: str(x),v) )).take(5)

[(120,
  '-0.04354475438594818, -0.3437068462371826, 0.08738048374652863, 0.1493648737668991, 0.011952451430261135, -0.09942986071109772, 0.2106037735939026, 0.3627055585384369, 0.1341874897480011, -0.15750350058078766, 0.28746142983436584, 0.2328723520040512, 0.017917631193995476, 0.03436492756009102, -0.07050832360982895, 0.23455484211444855, 0.011516330763697624, -0.056172922253608704, 0.10433101654052734, -0.019960220903158188, -0.07806042581796646, -0.1185324564576149, -0.11629392206668854, -0.03283442556858063, -0.18302929401397705, -0.5033071637153625, 0.07517089694738388, 0.03577866777777672, 0.2592799961566925, -0.08838721364736557, -0.03977544605731964, 0.12299688905477524, 0.005985778756439686, 0.024718027561903, 0.0628739520907402, 0.08207602798938751, -0.005692627746611834, -0.11079041659832001, 0.09192708879709244, 0.056098055094480515, 0.2521832585334778, 0.06714048236608505, 0.03959398716688156, -0.10585267096757889, -0.08365881443023682, 0.1313609480857849, 0.024214962

In [25]:
userID = 2093760 # we named this user Sara
recommendations = model.recommendProducts(userID, 5)

recommendations

[Rating(user=2093760, product=1034510, rating=0.3586525179541827),
 Rating(user=2093760, product=1008936, rating=0.3566350381522592),
 Rating(user=2093760, product=2000819, rating=0.3524946926006159),
 Rating(user=2093760, product=1300642, rating=0.3517005280211512),
 Rating(user=2093760, product=942, rating=0.35013897434053154)]

In [26]:
recommended_list = [1001819, 1003249, 1300642, 1007614, 892]

In [27]:
recommendedArtists = artistByID.filter(lambda artist: artist[0] in recommended_list).collect()

recommendedArtists

[(1007614, 'Jay-Z'),
 (1003249, 'Ludacris'),
 (1001819, '2Pac'),
 (892, 'Williams Traffic'),
 (1300642, 'The Game')]

#### Let's see what artist are Sara prefer

In [28]:
rawArtistsForUser = rawUserArtistData.map(lambda x: x.split(' '))\
                                     .filter(lambda x: int(x[0]) == userID)

In [29]:
existingProducts = rawArtistsForUser.map(lambda x: int(x[1])).collect()
existingProducts

[1180, 1255340, 378, 813, 942]

In [30]:
existingArtists = artistByID.filter(lambda artist: artist[0] in existingProducts).collect()
print('Sara prefers:')
for val in existingArtists:
    print(val)
    

Sara prefers:
(1180, 'David Gray')
(378, 'Blackalicious')
(813, 'Jurassic 5')
(1255340, 'The Saw Doctors')
(942, 'Xzibit')


#### Next step we need to analyze what relationship between existingArtists and recommened Artist.

In [31]:
'''----------------------------------------High level API---------------------------------------'''
'''Convert RDD(pipeline) to Spark-DF'''
from  pyspark.sql.types import Row

def f(x):
    d = {}
    for i in range(len(x)):
        d[str(i)] = x[i]
    return d

# EX
# seq = [['a','b','c'],
#        ['d','e','f'] ]
# print( f(seq) )
# print( Row(**f(seq)) )

In [32]:
def buildCount(rawUserArtistData, bArtistAlias):
    
    def getArtistCount(line):
        (userID, artistID, count) = map(lambda x: int(x), line.split(' '))    
        try:
            CorrectArtistID = bArtistAlias.value[artistID] # to map bad artist id to good artist id
        except KeyError:
            CorrectArtistID = artistID
        return (userID, CorrectArtistID, count)

    return rawUserArtistData.map(lambda line: getArtistCount(line))

In [33]:
# sc.broadcast use to spread parameter to all excecutor
# After we apply sc.broadcast, we can use .value method to see the dict value.
bArtistAlias = sc.broadcast(new_AA)
# bArtistAlias.value[1092764]
trainData_HLV = buildCount(UAD, bArtistAlias)

In [34]:
new_AD_df = new_AD.map(lambda x: Row(**f(x))).toDF()
mapping = dict(zip(['0', '1'], ['artist_id', 'artist_name']))
new_AD_df = new_AD_df.select([F.col(c).alias(mapping.get(c)) for c in new_AD_df.columns])

new_AD_df.show(5)

+---------+--------------------+
|artist_id|         artist_name|
+---------+--------------------+
|  1134999|        06Crazy Life|
|  6821360|        Pang Nakarin|
| 10113088|Terfel, Bartoli- ...|
| 10151459| The Flaming Sidebur|
|  6826647|   Bodenstandig 3000|
+---------+--------------------+
only showing top 5 rows



In [35]:
trainData_df = trainData_HLV.map(lambda x: Row(**f(x))).toDF()
mapping = dict(zip(['0', '1', '2'], ['user_id', 'artist_id', 'playcount']))
trainData_df = trainData_df.select([F.col(c).alias(mapping.get(c)) for c in trainData_df.columns]) # get 0,1,2 as Keys

trainData_df.show(5)

+-------+---------+---------+
|user_id|artist_id|playcount|
+-------+---------+---------+
|1000002|        1|       55|
|1000002|  1000006|       33|
|1000002|  1000007|        8|
|1000002|  1000009|      144|
|1000002|  1000010|      314|
+-------+---------+---------+
only showing top 5 rows



## The ALS Model in Spark contains the following helpful methods:

#### recommendForAllItems(int numUsers)
Returns top numUsers users recommended for each item, for all items.

#### recommendForAllUsers(int numItems)
Returns top numItems items recommended for each user, for all users.

#### recommendForItemSubset(Dataset<?> dataset, int numUsers)
Returns top numUsers users recommended for each item id in the input data set.

#### recommendForUserSubset(Dataset<?> dataset, int numItems)
Returns top numItems items recommended for each user id in the input data set.

In [36]:
userID = 2093760
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

model = ALS(implicitPrefs=True,
            rank=50,
            regParam=0.01, 
            alpha=40.0,
            maxIter=10,
            userCol="user_id", 
            itemCol="artist_id", 
            ratingCol="playcount")

# rank = number of features.

# # OR
# model = ALS()
# (alsEstimator.setRank(10)
#   .setUserCol("userid")
#   .setItemCol("artistid")
#   .setRatingCol("playcount")
#   .setMaxIter(5))

In [37]:
model_fit = model.fit(trainData_df)

In [38]:
# What Sara have been listened
trainData_df.filter(trainData_df['user_id'] == userID).show()

+-------+---------+---------+
|user_id|artist_id|playcount|
+-------+---------+---------+
|2093760|     1180|        1|
|2093760|  1255340|        3|
|2093760|      378|        1|
|2093760|      813|        2|
|2093760|      942|        7|
+-------+---------+---------+



In [39]:
model_fit.userFactors.show(1, truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                                                                                                                                                                                                                                                

In [40]:
# test
trainData_df.select(model.getUserCol()).distinct().limit(2).show()
# model.getUserCol() equivalent to trainData_df['user_id']

+-------+
|user_id|
+-------+
|1000061|
|1000070|
+-------+



In [41]:
trainData_df.createOrReplaceTempView('trainData_df_SQL')

In [42]:
# I prefer this one. It is far better T__T
sara = spark.sql('SELECT DISTINCT user_id \
           FROM trainData_df_SQL \
           WHERE user_id = 2093760'
         )

In [43]:
recom_user = model_fit.recommendForUserSubset(sara, 5)

In [44]:
recom_user.show(truncate=False)

+-------+----------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                 |
+-------+----------------------------------------------------------------------------------------------------------------+
|2093760|[[1008936, 0.3412131], [1034510, 0.3384863], [1004028, 0.3381605], [1300642, 0.33658555], [2000819, 0.33599353]]|
+-------+----------------------------------------------------------------------------------------------------------------+

